In [1]:
!pip3 install nltk

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 100.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 108.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 73.8 kB/s eta 0:00:00a 0:00:01


In [2]:
import torch
import torch.nn as nn

import nltk
import numpy as np
import pandas as pd
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

import json,urllib
from torch.utils.data import Dataset, DataLoader

import random

[nltk_data] Downloading package punkt to /home/rohan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax
        return out

In [4]:
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    """
    sentence = ["hello, "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag =   [0,     1,       0,   1,    0,      0,       0 ]
    """
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    
    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

In [11]:
#Three files : 
# 1.ubuntu_forum_data/Ubuntu-dialogue-corpus/dialogueText.csv -> 1038325 datapoints (116MB)
# 2.ubuntu_forum_data/Ubuntu-dialogue-corpus/dialogueText_196.csv -> 1048576 datapoints (996MB)
# 3.ubuntu_forum_data/Ubuntu-dialogue-corpus/dialogueText_301.csv -> 1048576 datapoints (1.8GB)

df = pd.read_csv("/media/rohan/DATA/AI_fellowship_2022/Course_2_DL/Ubuntu_chatbot/ubuntu_forum_data/Ubuntu-dialogue-corpus/dialogueText_301.csv")
df.head()

,folder,dialogueID,date,from,to,text
0,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,NaN,any ideas why java plugin takes so long to load?
1,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.4?
2,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,crimsun,yes
3,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.5 loads _much_ faster
4,301,1.tsv,2004-11-23T11:50:00.000Z,stuNNed,crimsun,noneus: how can i get 1.5 is there a .deb some...


In [23]:
conversation = df[['from','to','text']]
conversation.head()

,from,to,text
0,stuNNed,NaN,any ideas why java plugin takes so long to load?
1,crimsun,stuNNed,java 1.4?
2,stuNNed,crimsun,yes
3,crimsun,stuNNed,java 1.5 loads _much_ faster
4,stuNNed,crimsun,noneus: how can i get 1.5 is there a .deb some...


In [29]:
#extract all words from the ubuntu form dataset
all_words = []
x = []
for index, convo in conversation.iterrows():
    # print(convo["text"])
    for pattern in convo["text"] :
        w = tokenize(pattern)
        all_words.extend(w)
        x.append(w)
ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
print(all_words)

In [ ]:
X_train  = []
y_train = []
for pattern_sentence in x:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
y_train = X_train[1:]
X_train = X_train[:-1]

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train
        
    # dataset[idx]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

# Hyper paramters
batch_size = 8
hidden_size = 8
output_size = len(y_train[0])
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size = batch_size,
                        shuffle =False, num_workers=2)

device  = torch.device('cpu')
model = NeuralNet(input_size, hidden_size, output_size)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        # Forward
        outputs = model(words)
        loss = criterion(outputs, labels)

        # backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch +1) % 100 == 0:
        # f'epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}'
        print("epoch {}/{}, loss={:.4f}.".format(epoch+1,num_epochs,loss.item()))

# print(f'Final loss, loss={loss.item():.4f}')
print("Final Loss, loss{:.4f}".format(loss.item()))

In [ ]:
data = {
    "model_state":model.state_dict(),
    "input_size":input_size,
    "output_size":output_size,
    "hiddent_size": hidden_size,
    "all_words":all_words
}
FILE = "data.pth"
torch.save(data, FILE)

print("Traning complete. file saved to",FILE)

In [ ]:
# device  = torch.device('cpu')
# print(device)

# with open(url,'r') as f:
#   intents = json.load(f) 


# FILE = "data.pth"
# data = torch.load(FILE)

# input_size = data['input_size']
# hidden_size = data['hiddent_size']
# output_size = data['output_size']
# all_words = data['all_words']
# tags = data['tags']
# model_state = data['model_state']

# model = NeuralNet(input_size, hidden_size, output_size).to(device)
# model.load_state_dict(model_state)
# model.eval()

In [12]:
def callme():
  sen = input("Ubuntu_bro : Enter a query \n You: ")
  return sen

In [14]:
print(df)

bot_name = "Ubuntu_bro"

print("Namste !!! I am Ubuntu_bro. Just drop your Ubuntu related query ")
while True:
  sentence = input('You: ')

  if sentence =='quit':
    break
  
  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X)

  output = model(X)
  _, predicted = torch.max(output, dim=1)

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]

# 
# 

  print("{} I do not understand...".format(bot_name))

          folder dialogueID                      date       from       to  \
0            301      1.tsv  2004-11-23T11:49:00.000Z    stuNNed      NaN   
1            301      1.tsv  2004-11-23T11:49:00.000Z    crimsun  stuNNed   
2            301      1.tsv  2004-11-23T11:49:00.000Z    stuNNed  crimsun   
3            301      1.tsv  2004-11-23T11:49:00.000Z    crimsun  stuNNed   
4            301      1.tsv  2004-11-23T11:50:00.000Z    stuNNed  crimsun   
...          ...        ...                       ...        ...      ...   
16587825      32   1783.tsv  2007-11-15T03:38:00.000Z    koyo001      NaN   
16587826      32   1783.tsv  2007-11-15T03:39:00.000Z    koyo001      NaN   
16587827      32   1783.tsv  2007-11-15T03:39:00.000Z  neverblue      NaN   
16587828      32   1783.tsv  2007-11-15T03:40:00.000Z    koyo001   ikonia   
16587829      32   1783.tsv  2007-11-15T03:40:00.000Z  neverblue  koyo001   

                                                       text  
0          an